In [40]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer

In [72]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [52]:
stopwords = set(stopwords.words('english'))

In [3]:
test = pd.read_csv("test.csv")

In [4]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [5]:
test_labels = pd.read_csv("test_labels.csv")

In [6]:
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [7]:
train = pd.read_csv("train.csv")

In [8]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [12]:
for e in train["comment_text"][:5]:
    print(e)
    print("###############")

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
###############
D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)
###############
Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
###############
"
More
I can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any prefere

In [13]:
train.shape

# Flow:
2. Preprocess (lower, remove: emails, numbers, links, punctuation, stopwords
3. Lemmatize, Vectorize
4. Data selection
5. Models

In [55]:
def is_link(word):
    if "http" in word:
        return True
    return False

def is_email(unistr):
    password = unistr
    regex = '^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$'
    if re.search(regex, password):  
        return True
    else:  
        return False
    
def is_bignumber(word):
    return not re.search("d(1+)", word)

def is_aplha(word):
    test = ''.join([i for i in word if i.isalpha()])
    print(test)
    
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess(df):
    df["comment_text"] = df["comment_text"].apply(lambda x: x.lower())
    df["clean_comment"] = 0
    list_comments = []
    for idx, row in enumerate(df["comment_text"]):
        # print(f"Comment: {row}\n")
        text_split_list = row.split()
        list_word = []
        for idx, word in enumerate(text_split_list):
            # print(word)
            email = is_email(word)
            link = is_link(word)
            # number = is_bignumber(word)
            word = re.sub('[^a-zA-Z]+',"", word)
            flags = [email, link]
            if not any(flags) and word and word not in stopwords:
                list_word.append(lemmatizer.lemmatize(word, "v"))

        list_comments.append(" ".join(list_word))
#     print(list_comments)
    df["clean_comment"] = list_comments

preprocess(train)

<WordListCorpusReader in 'C:\\Users\\Plajer\\AppData\\Roaming\\nltk_data\\corpora\\stopwords'>

In [148]:

# lemmatizer.lemmatize("rocks")
sample_set2["comment_text"] = sample_set2["comment_text"].apply(lambda text: ' '.join(lemmatizer.lemmatize(i, "v") for i in text.lower().split()))

In [149]:
sample_set2

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
472,0132912330421bc5,oppose. think merge good idea.,0,0,0,0,0,0
498,014d00c8f2a76df4,interesting. check case number kunzang lhamo c...,0,0,0,0,0,0
499,014f50b1f448cb2d,transcluded part ga review seem update main ta...,0,0,0,0,0,0
545,016c95d317969274,jeff garcia gay not?,0,0,0,0,0,0
582,018942e044417b74,"sir imbecile, pervert.",1,0,0,0,1,0


In [154]:
string = "transcluded part ga review seem update main"
# print(string.lower().split())
print(lemmatizer.lemmatize("dogs", "v"))

dog


In [109]:
print(lemmatizer.lemmatize("better"))

better


In [178]:
X = vectorizer.fit_transform(sample_set2["comment_text"])
print(vectorizer.get_feature_names())
print(X.toarray())

['access', 'article', 'bernadino', 'can', 'case', 'check', 'cite', 'civil', 'click', 'content', 'correctly', 'criminal', 'date', 'either', 'find', 'further', 'ga', 'garcia', 'gay', 'get', 'go', 'good', 'guess', 'idea', 'imbecile', 'interesting', 'jeff', 'kunzang', 'lhamo', 'looong', 'main', 'may', 'merge', 'not', 'number', 'open', 'oppose', 'original', 'page', 'part', 'pervert', 'records', 'related', 'research', 'review', 'san', 'seem', 'since', 'sir', 'skim', 'sorry', 'step', 'stuff', 'subpage', 'talk', 'think', 'to', 'transcluded', 'up', 'update', 'use', 'version', 'wiki']
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0
  1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [1 0 1 2 2 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 2 1
  0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1]
 [0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0
  0 0 1 1 0 0 1 0 1 0 1 0 0 1 1 0 1 1 1 0 1 1 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0

In [161]:


vectorizer = CountVectorizer(min_df=0, lowercase=False)
lista1 = ["dupa", "nogi", "ręce"]
lista2 = ["stopy", "głowa", "palce"]

vectorizer.fit(lista1)
vectorizer.fit(lista2)
vectorizer.vocabulary_
# {'John': 0, 'chocolate': 1, 'cream': 2, 'hates': 3, 'ice': 4, 'likes': 5}

{'stopy': 2, 'głowa': 0, 'palce': 1}